In [2]:
from PIL import Image
import dlib
import cv2
import os
import numpy as np
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_curve, auc, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn import neighbors
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from keras.models import load_model  # TensorFlow is required for Keras to work
from PIL import Image, ImageOps  # Install pillow instead of PIL
import numpy as np

2023-12-10 12:30:36.390753: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-10 12:30:36.600085: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-10 12:30:37.704673: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
detector = dlib.get_frontal_face_detector()

In [5]:
# Replace this with the path to your image file with the appropriate extension (e.g., .jpg, .png)
image_path=r"FaceShape_Dataset/heart (1000).jpg"
image = Image.open(image_path).convert("RGB")

1st SNN

In [11]:
# Disable scientific notation for clarity 
np.set_printoptions(suppress=True)

from tensorflow.keras.models import load_model
  
# Load the model TEAM09/
model = load_model("converted_keras/keras_model.h5", compile=False)

# Load the labels
with open("converted_keras/labels.txt", "r") as file:
    class_names = file.readlines()


# Create the array of the right shape to feed into the keras model
# The 'length' or number of images you can put into the array is
# determined by the first position in the shape tuple, in this case 1

data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)

from PIL import Image

# resizing the image to be at least 224x224 and then cropping from the center
size = (224, 224)
image = ImageOps.fit(image, size, Image.Resampling.LANCZOS)

# turn the image into a numpy array
image_array = np.asarray(image)

# Normalize the image
normalized_image_array = (image_array.astype(np.float32) / 127.5) - 1

# Load the image into the array
data[0] = normalized_image_array

# Predicts the model
prediction = model.predict(data)
index = np.argmax(prediction)
class_name = class_names[index]
confidence_score = prediction[0][index]

# Print prediction and confidence score
print("Class:", class_name[2:], end="")
print("Confidence Score:", confidence_score)

2023-12-10 12:33:53.760501: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-12-10 12:33:53.863721: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-10 12:33:53.863975: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-10 12:33:53.863985: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:109] Couldn't get ptxas version : FAILED_PRECONDITION: Couldn't get ptxas/nvlink version string: INTERNAL: Couldn't invoke ptxas --version
2023-12-10 12:33:53.864161: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-12-10 12:33:53.864196: W tensorflow/compiler/xla/stream_executor/gpu/redzone_allocator.cc:318] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This mes

1/1 [==============================] - 1s 682ms/step
Class: Heart
Confidence Score: 0.9843203


2nd CNN

3rd KNN

In [13]:
knn_model_file = "KNN/knn_best_model.pkl"
best_model = joblib.load(knn_model_file)

/home/lisaa/mainpy/lib/python3.11/site-packages/sklearn/base.py:348: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.3.2 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [14]:
def extract_landmark_features(image_path):
    global noisy_datapoints

    landmark_features = []

    image = cv2.imread(image_path)
    image = cv2.resize(image, (500, 500))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    faces = detector(gray)

    if len(faces) != 1:
        noisy_datapoints += 1
        return None

    for face in faces:
        landmarks = predictor(gray, face)

        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y

            cv2.circle(gray, (x, y), 2, (0, 255, 0), -1)
            landmark_features.append((x, y))

    return landmark_features

In [9]:
# print("yes")
# x_image_paths = []
# print("1")
# test_path = "KNN/FaceShape_testing"
# print("2")
# for root, dirs, files in os.walk(test_path):
#     print("3")
#     for file in files:
#         print(file)
#         if file.endswith(".jpg") or file.endswith(".jpeg"):
#             label = file.split(" ")[0]
#             x_image_paths.append(os.path.join(root, file))
# x_predict = []
# y_labels = []
# for path in x_image_paths:
#     facepoints = extract_landmark_features(path)
#     if facepoints is None:
#         continue
#     x_predict.append(facepoints)
#     y_labels.append(path.split("\\")[1].split(".")[0])

# x_predict = np.array(x_predict)
# n_samples, n_landmarks, n_coordinates = x_predict.shape
# x_predict = x_predict.reshape(n_samples, n_landmarks * n_coordinates)

# x_predict = np.array(x_predict)
# y_predicted = best_model.predict(x_predict)
# y_predicted_proba = best_model.predict_proba(x_predict)

# for label, predict, prob in zip(y_labels, y_predicted, y_predicted_proba):
#     print(f"{label}: {predict} (Confidence: {prob.max():.2f})")

In [15]:
print("entered")
x_predict = []
print(image_path)
image=cv2.imread(image_path)
cv2.imshow("image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()
facepoints = extract_landmark_features(image_path)
# if facepoints is None:
#     print("None")
# x_predict.append(facepoints)

# x_predict = np.array(x_predict)
# n_samples, n_landmarks, n_coordinates = x_predict.shape
# x_predict = x_predict.reshape(n_samples, n_landmarks * n_coordinates)
# x_predict = np.array(x_predict)
# y_predicted = best_model.predict(x_predict)
# y_predicted_proba = best_model.predict_proba(x_predict)

# print(y_predicted)
# print(y_predicted_proba)
print("ok")

entered
FaceShape_Dataset/heart (1000).jpg
ok


4th SVM